With 1-dimensional CNN, Classify with the given raisin data!

In [74]:
import pandas as pd

raisin = pd.read_csv("https://raw.githubusercontent.com/KnightChaser/ML-challenge/main/EX_challenging_problems/02_raisin_classification/Raisin_Dataset.csv")
raisin.head()

,Area,MajorAxisLength,MinorAxisLength,Eccentricity,ConvexArea,Extent,Perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen


In [75]:
# Which type does the dataframe have?

print(pd.unique(raisin["Class"]))

['Kecimen' 'Besni']


In [76]:
print(raisin.isna())

      Area  MajorAxisLength  MinorAxisLength  Eccentricity  ConvexArea  \
0    False            False            False         False       False   
1    False            False            False         False       False   
2    False            False            False         False       False   
3    False            False            False         False       False   
4    False            False            False         False       False   
..     ...              ...              ...           ...         ...   
895  False            False            False         False       False   
896  False            False            False         False       False   
897  False            False            False         False       False   
898  False            False            False         False       False   
899  False            False            False         False       False   

     Extent  Perimeter  Class  
0     False      False  False  
1     False      False  False  
2     False    

In [77]:
raisin_input  = raisin[["Area", "MajorAxisLength", "MinorAxisLength", "Eccentricity", "ConvexArea", "Extent", "Perimeter"]].to_numpy()
raisin_target = raisin["Class"].to_numpy()

In [78]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

train_input, test_input, train_target, test_target = train_test_split(raisin_input, raisin_target, random_state = 0xCAFE, test_size = 0.2)

ss = StandardScaler()
ss.fit(train_input)

train_scaled = ss.transform(train_input)
test_scaled  = ss.transform(test_input)

In [79]:
print(train_scaled.shape)
print(test_scaled.shape)

(720, 7)
(180, 7)


In [81]:
import numpy as np

train_target = [1 if x == "Kecimen" else 0 for x in train_target]
test_target  = [1 if x == "Kecimen" else 0 for x in test_target]

print(train_target[:20])
print(test_target[:20])

train_target = np.array(train_target)
test_target  = np.array(test_target)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [86]:
from tensorflow import keras

def create_CNN_model(additional_layers = None):

    model = keras.Sequential()

    if additional_layers:
        for more_layer in additional_layers:
            model.add(more_layer)

    model.add(keras.layers.Dense(units = 2))        # output will be 2 types

    return model

In [112]:
# Build 1-dimensional CNN for classification

model = create_CNN_model([
    keras.layers.Conv1D(32, kernel_size = 2,                                    # Convolution (1D)
                            activation  = "relu",
                            padding     = "same",
                            input_shape = (train_scaled.shape[1], 1)),
    keras.layers.MaxPooling1D(pool_size = 2),                                   # Pooling (1D)
    # keras.layers.Conv1D(64, kernel_size = 2,
    #                         activation  = "relu",
    #                         padding     = "same"),
    # keras.layers.MaxPooling1D(),
    keras.layers.Flatten(),                                                     # Flatten
    keras.layers.Dense(units = 64, activation = "relu"),                        # ANN
    keras.layers.Dense(units = 32, activation = "relu"),                        # ANN (Cont'd)
    keras.layers.Dense(units = 16, activation = "softmax")
])

print(model.summary())


Model: "sequential_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_45 (Conv1D)          (None, 7, 32)             96        
                                                                 
 max_pooling1d_34 (MaxPoolin  (None, 3, 32)            0         
 g1D)                                                            
                                                                 
 flatten_27 (Flatten)        (None, 96)                0         
                                                                 
 dense_88 (Dense)            (None, 64)                6208      
                                                                 
 dense_89 (Dense)            (None, 32)                2080      
                                                                 
 dense_90 (Dense)            (None, 16)                528       
                                                     

In [114]:
## TRAIN!

# help wanted

from sklearn.utils import validation
from keras.optimizers import SGD

opt = SGD(learning_rate = 0.03)

model.compile(loss = "sparse_categorical_crossentropy",            # <--- CLASSIFICATION
              metrics = "accuracy",
              optimizer = "adam")

early_stopping_checkpoint = keras.callbacks.EarlyStopping(patience = 6, restore_best_weights = True, mode = "auto")
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5', save_best_only=True)

history = model.fit(train_scaled, train_target,
                     epochs = 20,
                     steps_per_epoch = 100,
                     validation_data = (test_scaled, test_target),
                     callbacks = [early_stopping_checkpoint, checkpoint_cb])

print()
print()
model.evaluate(test_scaled, test_target)

Epoch 1/20
100/100 [==============================] - 3s 9ms/step - loss: 6.2689e-07 - accuracy: 1.0000 - val_loss: 4.5035e-07 - val_accuracy: 1.0000
Epoch 2/20
100/100 [==============================] - 1s 7ms/step - loss: 3.8147e-07 - accuracy: 1.0000 - val_loss: 3.4107e-07 - val_accuracy: 1.0000
Epoch 3/20
100/100 [==============================] - 1s 7ms/step - loss: 2.9579e-07 - accuracy: 1.0000 - val_loss: 2.5564e-07 - val_accuracy: 1.0000
Epoch 4/20
100/100 [==============================] - 1s 7ms/step - loss: 2.4095e-07 - accuracy: 1.0000 - val_loss: 2.3776e-07 - val_accuracy: 1.0000
Epoch 5/20
100/100 [==============================] - 1s 6ms/step - loss: 2.3380e-07 - accuracy: 1.0000 - val_loss: 2.2650e-07 - val_accuracy: 1.0000
Epoch 6/20
100/100 [==============================] - 0s 5ms/step - loss: 2.0608e-07 - accuracy: 1.0000 - val_loss: 1.7815e-07 - val_accuracy: 1.0000
Epoch 7/20
100/100 [==============================] - 1s 5ms/step - loss: 1.6332e-07 - accuracy: 1.0

[1.1920928244535389e-07, 1.0]

In [90]:
print(train_scaled.shape)

(720, 7)
